# 데이터 불러오기

In [1]:
import json

file_name = '/workspace/train_data.json'

with open(file_name, "r", encoding='UTF-8') as read_file:
    data = json.load(read_file)

data

{'0': [[298, 991, 8, 133, 4, 12288],
  [182, 1139, 17, 534, 17, 16384],
  [191, 867, 14, 200, 7, 12288],
  [259, 817, 14, 200, 7, 8192],
  [243, 740, 8, 100, 4, 8192],
  [656, 793, 8, 100, 4, 8192],
  [463, 823, 6, 300, 3, 8192],
  [263, 714, 14, 200, 7, 8192],
  [219, 551, 12, 167, 6, 4096],
  [220, 454, 11, 333, 11, 4096],
  [354, 485, 10, 134, 5, 4096],
  [808, 464, 18, 267, 9, 4096],
  [817, 338, 14, 200, 7, 253952],
  [602, 383, 12, 167, 6, 253952],
  [459, 444, 6, 66, 3, 4096],
  [266, 489, 22, 333, 11, 4096],
  [218, 590, 12, 167, 6, 4096],
  [654, 629, 18, 267, 9, 4096],
  [481, 527, 20, 301, 10, 4096],
  [621, 507, 12, 367, 12, 4096],
  [335, 634, 14, 200, 7, 8192],
  [201, 950, 12, 367, 12, 12288],
  [221, 828, 26, 400, 13, 8192],
  [364, 1224, 17, 533, 17, 16384],
  [290, 1254, 11, 333, 11, 16384],
  [266, 699, 18, 266, 9, 8192],
  [160, 527, 20, 300, 10, 4096],
  [129, 414, 6, 67, 3, 258048],
  [204, 343, 16, 567, 8, 258048],
  [-87, 445, 6, 67, 3, 258048],
  [54, 575, 8, 1

In [2]:
len(data.keys())

162

In [3]:
print(type(data['4']))

<class 'list'>


# Torch

origin 폴더의 Seq2Seq.ipynb 파일을 옮겨가면서 시도중

In [4]:
# CUDA Availability
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
tup_list = []
for num, val in data.items():
    tup = (num, val)
    tup_list.append(tup)

tup_list.sort(key=lambda x: x[1], reverse=True)
mover_count = len(data)

In [17]:
import math
import random
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch import optim
from importlib import reload
import seq2seq as ss
reload(ss)

input_size = 6
output_size = 6
hidden_size = 256
mini_batch = 32
learning_rate=0.001

encoder = ss.EncoderMSE(input_size, hidden_size).to(device)
decoder = ss.DecoderMSE(input_size, hidden_size, output_size).to(device)
encoder_optimizer = optim.AdamW(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.AdamW(decoder.parameters(), lr=learning_rate)

criterion = nn.MSELoss()

In [27]:
packed_inputs = []
packed_inputs_d = []
batch_sizes = []
seq_sums = []

batch_count = math.ceil(len(data.keys()) / mini_batch)
for i in range(batch_count):
    batch_list = []
    seq_len = []
    seq_sum = 0
    for j in range(mini_batch):
        curr_idx = i * mini_batch + j
        if curr_idx < mover_count:
            m_id = tup_list[curr_idx][0]
            m_seq = data[m_id]
            m_tensor = torch.tensor(m_seq, dtype=torch.float, device=device)
            batch_list.append(m_tensor)
            seq_len.append(len(m_seq))
            seq_sum += len(m_seq)
            # print(m_tensor.shape)
    # print(seq_len)
    padded = pad_sequence(batch_list, batch_first=True)
    packed = pack_padded_sequence(padded, seq_len, batch_first=True, enforce_sorted=False)
    packed_inputs.append(packed)
    batch_sizes.append(len(seq_len))
    
    d_batch_list = []
    for t in batch_list:
        a = torch.zeros(1, input_size, device=device)
        # print(a)
        b = t[:-1, :]
        # print(b)
        c = torch.cat((a, b), 0)
        # print(c)
        d_batch_list.append(c)

    padded = pad_sequence(d_batch_list, batch_first=True)
    if padded.numel() == 0:
        # 빈 텐서를 패킹하려는 경우 패딩 없이 빈 텐서를 그대로 반환
        packed = padded
    else:
        packed = pack_padded_sequence(padded, seq_len, batch_first=True, enforce_sorted=False)
    packed_inputs_d.append(packed)
    seq_sums.append(seq_sum)
    
print(len(packed_inputs))
print(len(batch_sizes))
print(len(packed_inputs_d))
print(len(seq_sums))

RuntimeError: Cannot pack empty tensors.